<table width="100%" border="3">
  <tbody>
    <tr>
      <td ><img src="https://aidea-web.tw/images/web/logo_white.png" alt="Aidea" width="400"/></td>
      <td align='left'><h1>Exercise 2: Encode input</h1>s</td>
    </tr>
  </tbody>
</table>
Taxi Hotspot Prediction from input to output

In [ ]:
from google_drive_downloader import GoogleDriveDownloader
GoogleDriveDownloader.download_file_from_google_drive(file_id='13BhHdMYUP7VxYV1PNoQGx-wldEMebjyd',dest_path='./content', unzip=True)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Read training data
df_train = pd.read_csv('taxi_data/train_hire_stats.csv')
df_train.shape

In [ ]:
#Declare weekday
weekday = np.ones((len(df_train),), dtype=int)

In [ ]:
#Compute weekday & workday
#weekday()
#0 == Monday, #1 == Tuesday, #2 == Wednesday,  #3 == Thursday,  #4 == Friday,  #5 == Saturday, #6 == Sunday
from datetime import datetime

for index, row in df_train.iterrows():
    dd=datetime.strptime(row['Date'], "%Y-%m-%d")
    weekday[index]= dd.weekday() 



In [ ]:
#Build a new dataframe from the training data
RawX = pd.DataFrame(df_train[["Zone_ID", "Hour_slot"]])
RawX['weekday'] = weekday

In [ ]:
#See the raw input data
RawX.head()

In [ ]:
#Use OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [ ]:
#Build encoder
enc.fit_transform(RawX)
enc.categories_

In [ ]:
#Transform data into one hot vector
X = enc.transform(RawX).toarray()
X.shape

In [ ]:
#See the cooked input data
X[0:3, :]

In [ ]:
#Resize y into (0,1)
y = df_train["Hire_count"].values/339.0
y.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout

In [ ]:
# create and fit a Baseline Neural Network Model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=56))
model.add(Dense(units=10, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

In [ ]:
model.compile(loss='mse', optimizer='adam')
model.fit(X, y, epochs=10, batch_size=1024, verbose=1)

In [ ]:
df_test = pd.read_csv('taxi_data/test_hire_stats.csv')
df_test.shape

In [ ]:
#Declare weekday & workday
weekday2 = np.ones((len(df_test),), dtype=int)

In [ ]:
for index, row in df_test.iterrows():
    dd=datetime.strptime(row['Date'], "%Y-%m-%d")
    weekday2[index]= dd.weekday() 

In [ ]:
Test = pd.DataFrame(df_test[["Zone_ID", "Hour_slot"]])
Test['weekday'] = weekday2

In [ ]:
Xtest = enc.transform(Test).toarray()
Xtest.shape

In [ ]:
yt = model.predict(Xtest)*339.0

In [ ]:
plt.plot(yt)

In [ ]:
dfOut = pd.DataFrame(yt)

In [ ]:
dfOut.to_csv('Result2.csv')